# RAG with Vector Search

In [38]:
import os

from dotenv import load_dotenv
from openai import OpenAI
from qdrant_client import QdrantClient, models
import requests

load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_ENDPOINT = os.getenv("OPENROUTER_ENDPOINT")
OPENROUTER_MODEL = os.getenv("OPENROUTER_MODEL")

ai_client = OpenAI(
  base_url=OPENROUTER_ENDPOINT,
  api_key=OPENROUTER_API_KEY,
)

EMBEDDING_DIMENSIONALITY = 512
collection_name = "zoomcamp-faq"
model_handle = "jinaai/jina-embeddings-v2-small-en"
qd_client = QdrantClient("http://localhost:6333")

In [39]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [40]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [41]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [42]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [43]:
def search(query, limit=5, course = 'data-engineering-zoomcamp'):
    print(f'✅ vector_search is used in {collection_name}')
    resp = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    results = []

    for point in resp.points:
        results.append(point.payload)

    return results

def replace_placeholders(template_path, context_content, question_content):
    try:
        with open(template_path, "r") as f:
            prompt = f.read()

        prompt = prompt.replace("{{CONTEXT}}", context_content)
        prompt = prompt.replace("{{QUESTION}}", question_content)

        return prompt
    except FileNotFoundError:
        print(f"Error: Template file not found at {template_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def prepare_prompt(question):
    context_data = ""
    results = search(question)
    if not results:
        return None
    for item in results:

        context_data = context_data + f"section: {item['section']}\nanswer: {item['text']}\n\n"

    prompt = replace_placeholders("prompt_assistant.txt", context_data, question)

    return prompt

def rag(question):
    prompt = prepare_prompt(question)
    if not prompt:
        print("Unable to find information about the question.")

    response = ai_client.chat.completions.create(
        model=OPENROUTER_MODEL,
        messages=[
            {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [44]:
result = rag("What if I submit homeworks late?")
print(result)

✅ vector_search is used in zoomcamp-faq
<answer>
Hi there!

Regarding late homework submissions, the FAQ states that generally, late submissions are not allowed. However, if the submission form is still open after the due date, you may still be able to submit your homework. You should always confirm your submission by checking the date-timestamp on the Course page.

I hope this helps!

Best,
Your TA
</answer>
